In [1]:
import hashlib

class Node:
    def __init__(self, node_id):
        self.id = node_id

class KBucket:
    def __init__(self, depth, capacity=3, range_start=0, range_end=(1 << 160) - 1):
        self.nodes = []
        self.capacity = capacity
        self.depth = depth
        self.range_start = range_start
        self.range_end = range_end

    def insert_node(self, node_id):
        if len(self.nodes) < self.capacity:
            self.nodes.append(Node(node_id))
        else:
            if self.node_in_bucket_range(node_id):
                self.split_bucket()
                self.insert_node(node_id)
            else:
                # 节点ID不在当前桶范围，丢弃
                print(f"Node {node_id} outside of bucket range, discarded.")

    def node_in_bucket_range(self, node_id):
        # 判断节点ID是否在当前桶的范围内
        node_id_num = int(node_id, 16)
        return self.range_start <= node_id_num <= self.range_end

    def split_bucket(self):
        mid_point = (self.range_start + self.range_end) // 2
        left_bucket = KBucket(self.depth + 1, self.capacity, self.range_start, mid_point)
        right_bucket = KBucket(self.depth + 1, self.capacity, mid_point + 1, self.range_end)
        
        # 重新分配当前桶中的所有节点
        for node in self.nodes:
            if self.range_start <= int(node.id, 16) <= mid_point:
                left_bucket.nodes.append(node)
            else:
                right_bucket.nodes.append(node)
        
        # 为了简化，只保留一个桶
        self.nodes = []
        self.nodes.extend(left_bucket.nodes if len(left_bucket.nodes) <= len(right_bucket.nodes) else right_bucket.nodes)
        self.range_start = left_bucket.range_start if len(left_bucket.nodes) <= len(right_bucket.nodes) else right_bucket.range_start
        self.range_end = left_bucket.range_end if len(left_bucket.nodes) <= len(right_bucket.nodes) else right_bucket.range_end

        print("Bucket split into two new buckets")

    def printBucketContents(self):
        for node in self.nodes:
            print(node.id)

# 示例
bucket = KBucket(depth=0)
bucket.insert_node('001')
bucket.insert_node('002')
bucket.insert_node('003')
bucket.insert_node('004')  # 触发分裂
bucket.printBucketContents()


Bucket split into two new buckets
004


In [2]:
class TestKBucket:
    def __init__(self):
        # 初始化一个桶，假定深度为0，容量为3
        self.bucket = KBucket(depth=0, capacity=3)

    def test_insertion(self):
        print("Testing basic insertion...")
        self.bucket.insert_node('001')
        self.bucket.insert_node('002')
        assert len(self.bucket.nodes) == 2, "Insertion failed!"
        print("Basic insertion passed.")

    def test_bucket_split(self):
        print("Testing bucket split...")
        self.bucket.insert_node('003')
        # 第四个节点应触发分裂
        self.bucket.insert_node('004')
        # 根据实际逻辑，节点应该仍然存在于一个新桶中
        assert len(self.bucket.nodes) > 0, "Bucket did not manage nodes correctly."
        print(f"Bucket split passed, remaining nodes: {len(self.bucket.nodes)}")

    def test_node_discard(self):
        print("Testing node discard...")
        # 填充桶，以确保桶已满
        self.bucket.insert_node('001')
        self.bucket.insert_node('002')
        self.bucket.insert_node('003')
        # print(f"Initial bucket nodes: {[node.id for node in self.bucket.nodes]}")
        
        original_node_in_bucket_range = self.bucket.node_in_bucket_range
        # 重写桶范围检测逻辑以模拟节点不在范围内的情况
        self.bucket.node_in_bucket_range = lambda x: False
        self.bucket.insert_node('005')
        # 恢复原来的范围检测逻辑
        self.bucket.node_in_bucket_range = original_node_in_bucket_range
        # print(f"Bucket nodes after discard attempt: {[node.id for node in self.bucket.nodes]}")
        
        assert len(self.bucket.nodes) == 3, "Node discard failed."
        print("Node discard passed.")

    def test_print_contents(self):
        print("Testing print contents...")
        # 手动添加节点，用于测试打印
        self.bucket.nodes = [Node('010'), Node('011'), Node('012')]
        self.bucket.print_contents()
        print("Print contents test complete. Check output manually.")

# 创建测试实例
test = TestKBucket()
test.test_insertion()
test.test_bucket_split()
test.test_node_discard()
test.test_print_contents()


Testing basic insertion...
Basic insertion passed.
Testing bucket split...
Bucket split into two new buckets
Bucket split passed, remaining nodes: 1
Testing node discard...
Node 003 outside of bucket range, discarded.
Node 005 outside of bucket range, discarded.
Node discard passed.
Testing print contents...
010
011
012
Print contents test complete. Check output manually.


In [3]:
# 模拟多节点
import hashlib
import random

class Node:
    def __init__(self, node_id):
        self.id = node_id

class KBucket:
    def __init__(self, depth, capacity=3, range_start=0, range_end=(1 << 160) - 1):
        self.nodes = []
        self.capacity = capacity
        self.depth = depth
        self.range_start = range_start
        self.range_end = range_end

    def insert_node(self, node_id):
        if len(self.nodes) < self.capacity:
            self.nodes.append(Node(node_id))
        else:
            if self.node_in_bucket_range(node_id):
                self.split_bucket()
                self.insert_node(node_id)
            else:
                print(f"Node {node_id} outside of bucket range, discarded.")

    def node_in_bucket_range(self, node_id):
        node_id_num = int(node_id, 16)
        return self.range_start <= node_id_num <= self.range_end

    def split_bucket(self):
        mid_point = (self.range_start + self.range_end) // 2
        left_bucket = KBucket(self.depth + 1, self.capacity, self.range_start, mid_point)
        right_bucket = KBucket(self.depth + 1, self.capacity, mid_point + 1, self.range_end)
        for node in self.nodes:
            if self.range_start <= int(node.id, 16) <= mid_point:
                left_bucket.nodes.append(node)
            else:
                right_bucket.nodes.append(node)
        self.nodes = []
        self.nodes.extend(left_bucket.nodes if len(left_bucket.nodes) <= len(right_bucket.nodes) else right_bucket.nodes)
        self.range_start = left_bucket.range_start if len(left_bucket.nodes) <= len(right_bucket.nodes) else right_bucket.range_start
        self.range_end = left_bucket.range_end if len(left_bucket.nodes) <= len(right_bucket.nodes) else right_bucket.range_end
        print("Bucket split into two new buckets")

    def find_node(self, node_id):
        for node in self.nodes:
            if node.id == node_id:
                return [node.id]
        # Node not found, return 2 random nodes if possible
        return [node.id for node in random.sample(self.nodes, min(2, len(self.nodes)))]

    def print_contents(self):
        for node in self.nodes:
            print(node.id)

class Peer:
    def __init__(self, node_id):
        self.id = node_id
        self.bucket = KBucket(depth=0)

    def broadcast_new_node(self, new_node_id):
        self.bucket.insert_node(new_node_id)

def initialize_network():
    initial_peers = [Peer(f'{i:03}') for i in range(1, 6)]
    new_peers = [f'{i:03}' for i in range(6, 206)]
    for new_peer_id in new_peers:
        random.choice(initial_peers).broadcast_new_node(new_peer_id)

    for peer in initial_peers:
        print(f"Peer {peer.id} bucket contents:")
        peer.bucket.print_contents()

initialize_network()


Bucket split into two new buckets
Bucket split into two new buckets
Bucket split into two new buckets
Node 027 outside of bucket range, discarded.
Bucket split into two new buckets
Node 034 outside of bucket range, discarded.
Node 035 outside of bucket range, discarded.
Node 036 outside of bucket range, discarded.
Bucket split into two new buckets
Node 038 outside of bucket range, discarded.
Node 039 outside of bucket range, discarded.
Node 040 outside of bucket range, discarded.
Node 043 outside of bucket range, discarded.
Node 044 outside of bucket range, discarded.
Node 045 outside of bucket range, discarded.
Node 046 outside of bucket range, discarded.
Node 047 outside of bucket range, discarded.
Node 048 outside of bucket range, discarded.
Node 049 outside of bucket range, discarded.
Node 050 outside of bucket range, discarded.
Node 051 outside of bucket range, discarded.
Node 052 outside of bucket range, discarded.
Node 053 outside of bucket range, discarded.
Node 054 outside of 

In [1]:
# 实验二

class KBucket:
    def __init__(self, depth, capacity=160, range_start=0, range_end=(1 << 160) - 1):
        self.peers = []
        self.capacity = capacity
        self.depth = depth
        self.range_start = range_start
        self.range_end = range_end

    def insert_peer(self, peer):
        if len(self.peers) < self.capacity:
            self.peers.append(peer)
        else:
            if self.peer_in_bucket_range(peer.id):
                self.split_bucket()
                self.insert_peer(peer)
            else:
                print(f"Peer {peer.id} outside of bucket range, discarded.")

    def peer_in_bucket_range(self, peer_id):
        peer_id_num = int(peer_id, 16)
        return self.range_start <= peer_id_num <= self.range_end

    def split_bucket(self):
        mid_point = (self.range_start + self.range_end) // 2
        left_bucket = KBucket(self.depth + 1, self.capacity, self.range_start, mid_point)
        right_bucket = KBucket(self.depth + 1, self.capacity, mid_point + 1, self.range_end)
        for peer in self.peers:
            if self.range_start <= int(peer.id, 16) <= mid_point:
                left_bucket.peers.append(peer)
            else:
                right_bucket.peers.append(peer)
        self.peers = []
        self.peers.extend(left_bucket.peers if len(left_bucket.peers) <= len(right_bucket.peers) else right_bucket.peers)
        self.range_start = left_bucket.range_start if len(left_bucket.peers) <= len(right_bucket.peers) else right_bucket.range_start
        self.range_end = left_bucket.range_end if len(left_bucket.peers) <= len(right_bucket.peers) else right_bucket.range_end

    def find_closest_peers(self, key, count=2):
        # Simple placeholder to return closest peers; actual implementation would use a more sophisticated metric
        return sorted(self.peers, key=lambda peer: abs(int(peer.id, 16) - int(key, 16)))[:count]

    def printBucketContents(self):
        for peer in self.peers:
            print(peer.id)


import hashlib

class Peer:
    def __init__(self, node_id):
        self.id = node_id
        self.bucket = KBucket(depth=0)
        self.storage = {}

    def set_value(self, key, value):
        key_hash = hashlib.sha1(key).hexdigest()
        if key_hash != value.hex():
            return False
        if key in self.storage:
            return True
        self.storage[key] = value
        closest_peers = self.bucket.find_closest_peers(key_hash)
        for peer in closest_peers:
            peer.set_value(key, value)
        return True

    def get_value(self, key):
        if key in self.storage:
            return self.storage[key]
        closest_peers = self.bucket.find_closest_peers(key)
        for peer in closest_peers:
            value = peer.get_value(key)
            if value:
                return value
        return None

    def broadcast_new_peer(self, new_peer):
        self.bucket.insert_peer(new_peer)




In [1]:
# 有问题的实现
import hashlib
import random
import string

class KBucket:
    def __init__(self, depth, capacity=160, range_start=0, range_end=(1 << 160) - 1):
        self.peers = []
        self.capacity = capacity
        self.depth = depth
        self.range_start = range_start
        self.range_end = range_end

    def insert_peer(self, peer):
        if len(self.peers) < self.capacity:
            self.peers.append(peer)
        else:
            if self.peer_in_bucket_range(peer.id):
                self.split_bucket()
                self.insert_peer(peer)
            else:
                print(f"Peer {peer.id} outside of bucket range, discarded.")

    def peer_in_bucket_range(self, peer_id):
        peer_id_num = int(peer_id, 16)
        return self.range_start <= peer_id_num <= self.range_end

    def split_bucket(self):
        mid_point = (self.range_start + self.range_end) // 2
        left_bucket = KBucket(self.depth + 1, self.capacity, self.range_start, mid_point)
        right_bucket = KBucket(self.depth + 1, self.capacity, mid_point + 1, self.range_end)
        for peer in self.peers:
            if self.range_start <= int(peer.id, 16) <= mid_point:
                left_bucket.peers.append(peer)
            else:
                right_bucket.peers.append(peer)
        self.peers = []
        self.peers.extend(left_bucket.peers if len(left_bucket.peers) <= len(right_bucket.peers) else right_bucket.peers)
        self.range_start = left_bucket.range_start if len(left_bucket.peers) <= len(right_bucket.peers) else right_bucket.range_start
        self.range_end = left_bucket.range_end if len(left_bucket.peers) <= len(right_bucket.peers) else right_bucket.range_end

    def find_closest_peers(self, key, count=2):
        return sorted(self.peers, key=lambda peer: abs(int(peer.id, 16) - int(key, 16)))[:count]

class Peer:
    def __init__(self, node_id):
        self.id = node_id
        self.bucket = KBucket(depth=0)
        self.storage = {}

    def set_value(self, key, value, depth=0):
        if depth > 3:  # 限制递归深度为3
            return False
        key_hash = hashlib.sha1(key.encode()).hexdigest()
        if key_hash != hashlib.sha1(value.encode()).hexdigest():
            return False
        if key in self.storage:
            return True
        self.storage[key] = value
        closest_peers = self.bucket.find_closest_peers(key_hash)
        for peer in closest_peers:
            peer.set_value(key, value, depth + 1)  # 递增递归深度
        return True

    def get_value(self, key):
        if key in self.storage:
            return self.storage[key]
        closest_peers = self.bucket.find_closest_peers(key)
        for peer in closest_peers:
            value = peer.get_value(key)
            if value:
                return value
        return None

def main():
    # Initialize 100 peers
    peers = [Peer(f'{i:03}') for i in range(100)]

    # Generate 200 random strings and their hashes
    keys = []
    for _ in range(200):
        random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(5, 20)))
        key = hashlib.sha1(random_string.encode()).hexdigest()
        keys.append(key)
        chosen_peer = random.choice(peers)
        chosen_peer.set_value(key, random_string)

    # Retrieve 100 random keys
    retrieved_values = []
    for key in random.sample(keys, 100):
        chosen_peer = random.choice(peers)
        value = chosen_peer.get_value(key)
        retrieved_values.append(value)

    # Print retrieved values
    print(retrieved_values)

if __name__ == '__main__':
    main()


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [16]:
# 简化了KBucket

import hashlib
import random
import string

def calculate_distance(id1, id2):
    return int(id1, 16) ^ int(id2, 16)

class KBucket: # 用实验一的实现有点复杂没能跑通
    def __init__(self, capacity=160):
        self.peers = []
        self.capacity = capacity

    def insert_peer(self, peer):
        if len(self.peers) < self.capacity:
            self.peers.append(peer)

    def find_closest_peers(self, key, count=5):
        return sorted(self.peers, key=lambda peer: calculate_distance(peer.id, key))[:count]

class DHT:
    def __init__(self):
        self.buckets = [KBucket() for _ in range(160)]

class Peer:
    def __init__(self, node_id, dht):
        self.id = node_id
        self.dht = dht
        self.storage = {}

    def set_value(self, key, value, depth=0, tried_peers=None):
        if depth > 10 or tried_peers and self.id in tried_peers:
            return False
        tried_peers = tried_peers or set()
        tried_peers.add(self.id)

        key_hash = hashlib.sha1(key).hexdigest()
        self.storage[key] = value
        bucket_index = self.calculate_bucket_index(key_hash)
        closest_peers = self.dht.buckets[bucket_index].find_closest_peers(key_hash)
        for peer in closest_peers:
            if peer.id not in tried_peers:
                peer.set_value(key, value, depth + 1, tried_peers)
        return True

    def get_value(self, key):
        if key in self.storage:
            return self.storage[key]
        key_hash = hashlib.sha1(key).hexdigest()
        bucket_index = self.calculate_bucket_index(key_hash)
        closest_peers = self.dht.buckets[bucket_index].find_closest_peers(key_hash)
        for peer in closest_peers:
            value = peer.get_value(key)
            if value:
                return value
        return None

    def calculate_bucket_index(self, key_hash):
        combined_hash = hashlib.sha1((self.id + key_hash).encode()).hexdigest()
        return int(combined_hash, 16) % 160

def main():
    dht = DHT()
    peers = [Peer(f'{i:03}', dht) for i in range(100)]
    for peer in peers:
        initial_bucket_index = int(hashlib.sha1(peer.id.encode()).hexdigest(), 16) % 160
        dht.buckets[initial_bucket_index].insert_peer(peer)

    for _ in range(20): # 200太大了好像会崩溃
        random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
        key = random_string.encode()
        value = hashlib.sha1(key).digest()
        chosen_peer = random.choice(peers)
        chosen_peer.set_value(key, value)

    for _ in range(10): # 对应的取到了10个
        chosen_peer = random.choice(peers)
        if chosen_peer.storage:
            key = random.choice(list(chosen_peer.storage.keys()))
            value = chosen_peer.get_value(key)
            if value:
                print(f"Value retrieved: {value}")
            else:
                print("Value retrieval failed.")
        else:
            print(f"No values stored for peer {chosen_peer.id}")

if __name__ == '__main__':
    main()



Value retrieved: b'Z`\\\xf2\xd5}2w\xab\xe4\x93|\xb2\x8a\xd0,\xa5\x89\xf91'
Value retrieved: b'\xd2<\xf7\x9b\xd3\xad\x03p\xd9\xae4\xe4\xed\xb4;\xd5S+1\xbf'
No values stored for peer 015
Value retrieved: b'\xab\xed0\x17\xb8\xae\xe7\x01\xaa\x01Q\x1ec\xaa\xbb\x92a\xfb\x99G'
Value retrieved: b'\xb0`\x9b$;:\x9dWQ\x98\x8b\x96\x92\xec\xf7\xb2B\xa1s\xa5'
No values stored for peer 091
Value retrieved: b'Bk\x04\xd8\x08WB\xa4H\xe6\xc8`\x96\xe1\x88\xc2(y\xe4\xdc'
Value retrieved: b'\xb0`\x9b$;:\x9dWQ\x98\x8b\x96\x92\xec\xf7\xb2B\xa1s\xa5'
Value retrieved: b'\xb0`\x9b$;:\x9dWQ\x98\x8b\x96\x92\xec\xf7\xb2B\xa1s\xa5'
No values stored for peer 004


In [4]:
# Dev版本,尚未完全实现

import hashlib
import random
import string

def calculate_distance(id1, id2):
    return int(id1, 16) ^ int(id2, 16)

class KBucket:
    def __init__(self, depth, capacity=160, range_start=0, range_end=(1 << 160) - 1):
        self.peers = []
        self.capacity = capacity
        self.depth = depth
        self.range_start = range_start
        self.range_end = range_end

    def insert_peer(self, peer):
        if len(self.peers) < self.capacity:
            self.peers.append(peer)
        else:
            if self.peer_in_bucket_range(peer.id):
                self.split_bucket()
                self.insert_peer(peer)
            else:
                print(f"Peer {peer.id} outside of bucket range, discarded.")

    def peer_in_bucket_range(self, peer_id):
        peer_id_num = int(peer_id, 16)
        return self.range_start <= peer_id_num <= self.range_end

    def split_bucket(self):
        mid_point = (self.range_start + self.range_end) // 2
        left_bucket = KBucket(self.depth + 1, self.capacity, self.range_start, mid_point)
        right_bucket = KBucket(self.depth + 1, self.capacity, mid_point + 1, self.range_end)
        for peer in self.peers:
            if self.range_start <= int(peer.id, 16) <= mid_point:
                left_bucket.peers.append(peer)
            else:
                right_bucket.peers.append(peer)
        self.peers = []
        self.peers.extend(left_bucket.peers if len(left_bucket.peers) <= len(right_bucket.peers) else right_bucket.peers)
        self.range_start = left_bucket.range_start if len(left_bucket.peers) <= len(right_bucket.peers) else right_bucket.range_start
        self.range_end = left_bucket.range_end if len(left_bucket.peers) <= len(right_bucket.peers) else right_bucket.range_end

    def find_closest_peers(self, key, count=2):
        return sorted(self.peers, key=lambda peer: abs(int(peer.id, 16) - int(key, 16)))[:count]

class DHT:
    def __init__(self):
        self.buckets = [KBucket(depth=0, capacity=160, range_start=0, range_end=(1 << 160) - 1)]

    def find_bucket_for_key(self, key_hash):
        for bucket in self.buckets:
            if bucket.peer_in_bucket_range(key_hash):
                return bucket
        return None

    def insert_peer(self, peer):
        peer_id_num = int(peer.id, 16)
        bucket = self.find_bucket_for_key(peer.id)
        if bucket:
            bucket.insert_peer(peer)
        else:
            print(f"Peer {peer.id} could not be placed in any bucket.")

class Peer:
    def __init__(self, node_id, dht):
        self.id = node_id
        self.dht = dht
        self.storage = {}

    def set_value(self, key, value, depth=0, tried_peers=None):
        if depth > 10 or (tried_peers and self.id in tried_peers):
            return False
        tried_peers = tried_peers or set()
        tried_peers.add(self.id)

        key_hash = hashlib.sha1(key).hexdigest()
        self.storage[key] = value
        bucket = self.dht.find_bucket_for_key(key_hash)
        if bucket:
            closest_peers = bucket.find_closest_peers(key_hash)
            for peer in closest_peers:
                if peer.id not in tried_peers:
                    peer.set_value(key, value, depth + 1, tried_peers)
            return True
        return False

    def get_value(self, key):
        key_hash = hashlib.sha1(key).hexdigest()
        bucket = self.dht.find_bucket_for_key(key_hash)
        if bucket:
            closest_peers = bucket.find_closest_peers(key_hash)
            for peer in closest_peers:
                value = peer.get_value(key)
                if value:
                    return value
        return None


def main():
    dht = DHT()
    # Initialize buckets
    initial_range = (1 << 160) - 1
    initial_buckets = [KBucket(depth=0, capacity=160, range_start=0, range_end=initial_range)]
    dht.buckets = initial_buckets
    
    # Initialize peers
    peers = [Peer(f'{i:03}', dht) for i in range(100)]
    for peer in peers:

        placed = False
        for bucket in dht.buckets:
            if bucket.peer_in_bucket_range(peer.id):
                bucket.insert_peer(peer)
                placed = True
                break
        if not placed:
            print(f"Peer {peer.id} could not be placed in any bucket.")

    # Set and retrieve values
    for _ in range(20):
        random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
        key = random_string.encode()
        value = hashlib.sha1(key).digest()
        chosen_peer = random.choice(peers)
        chosen_peer.set_value(key, value)

    for _ in range(10):
        chosen_peer = random.choice(peers)
        if chosen_peer.storage:
            key = random.choice(list(chosen_peer.storage.keys()))
            value = chosen_peer.get_value(key)
            if value:
                print(f"Value retrieved: {value}")
            else:
                print("Value retrieval failed.")
        else:
            print(f"No values stored for peer {chosen_peer.id}")

if __name__ == '__main__':
    main()



No values stored for peer 026
No values stored for peer 036
No values stored for peer 021
No values stored for peer 086


RecursionError: maximum recursion depth exceeded while calling a Python object